In [1]:
!pip install transformers datasets accelerate peft
!pip install -U bitsandbytes

import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments, BitsAndBytesConfig
from datasets import load_dataset
import time
from peft import get_peft_model, LoraConfig
from transformers import DataCollatorWithPadding

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 7.2 MB/s eta 0:00:00


In [2]:
# Load SNLI dataset
dataset = load_dataset("snli")

# Select samples for training, validation, and testing
train_dataset = dataset['train'].select(range(0, 550000, 550))  # Every 550th sample
val_dataset = dataset['validation'].select(range(0, 10000, 100))  # Every 100th sample
test_dataset = dataset['test'].select(range(0, 10000, 100))  # Every 100th sample

# Print dataset sizes
print(f"Training samples: {len(train_dataset)}")
print(f"Validation samples: {len(val_dataset)}")
print(f"Test samples: {len(test_dataset)}")

README.md:   0%|          | 0.00/16.0k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/412k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/413k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/19.6M [00:00<?, ?B/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/550152 [00:00<?, ? examples/s]

Training samples: 1000
Validation samples: 100
Test samples: 100


In [3]:
# Load the Phi2 model and tokenizer from Hugging Face
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_use_double_quant=False,
)

model = AutoModelForSequenceClassification.from_pretrained(
    "microsoft/phi-2",
    num_labels=3,
    trust_remote_code=True,
    torch_dtype=torch.bfloat16,
    quantization_config=bnb_config
)

config.json:   0%|          | 0.00/735 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/35.7k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of PhiForSequenceClassification were not initialized from the model checkpoint at microsoft/phi-2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2", trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/7.34k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

In [6]:
# Tokenize the datasets with padding
def preprocess_function(examples):
    encoding = tokenizer(
        examples['premise'],
        examples['hypothesis'],
        truncation=True,
        padding="max_length",  # Pad to max length
        max_length=512         # Set a max length
    )
    encoding['label'] = examples['label']  # Ensure that label is included

    # Debugging: Print the encoding structure to confirm correct keys
    print(f"Encoding keys: {list(encoding.keys())}")
    return encoding
# Filter out any samples with unexpected label values
def filter_labels(dataset, allowed_labels=[0, 1, 2]):
    return dataset.filter(lambda example: example['label'] in allowed_labels)

# Apply filter to train, validation, and test datasets
train_dataset = filter_labels(train_dataset)
val_dataset = filter_labels(val_dataset)
test_dataset = filter_labels(test_dataset)

# Print dataset sizes after filtering
print(f"Filtered Training samples: {len(train_dataset)}")
print(f"Filtered Validation samples: {len(val_dataset)}")
print(f"Filtered Test samples: {len(test_dataset)}")

# Tokenize datasets
train_tokenized = train_dataset.map(preprocess_function, batched=True)
val_tokenized = val_dataset.map(preprocess_function, batched=True)
test_tokenized = test_dataset.map(preprocess_function, batched=True)
# Confirm the presence of columns before setting format
# Debugging: Verify that 'input_ids', 'attention_mask', and 'label' are present
print(f"Columns in train_tokenized: {train_tokenized.column_names}")
print(f"Sample entry in train_tokenized: {train_tokenized[0]}")
print(f"Keys in sample entry: {list(train_tokenized[0].keys())}")
print(f"Train tokenized format before setting format: {train_tokenized.column_names}")
# Set the format for PyTorch
train_tokenized.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
val_tokenized.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
test_tokenized.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])


Filter:   0%|          | 0/1000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/99 [00:00<?, ? examples/s]

Filter:   0%|          | 0/100 [00:00<?, ? examples/s]

Filtered Training samples: 1000
Filtered Validation samples: 99
Filtered Test samples: 100


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Encoding keys: ['input_ids', 'attention_mask', 'label']


Map:   0%|          | 0/99 [00:00<?, ? examples/s]

Encoding keys: ['input_ids', 'attention_mask', 'label']


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Encoding keys: ['input_ids', 'attention_mask', 'label']
Columns in train_tokenized: ['premise', 'hypothesis', 'label', 'input_ids', 'attention_mask']
Sample entry in train_tokenized: {'premise': 'A person on a horse jumps over a broken down airplane.', 'hypothesis': 'A person is training his horse for a competition.', 'label': 1, 'input_ids': [32, 1048, 319, 257, 8223, 18045, 625, 257, 5445, 866, 19401, 13, 32, 1048, 318, 3047, 465, 8223, 329, 257, 5449, 13, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256

In [7]:
# Ensure that the data types for input_ids and attention_mask are int64 and that labels are within range [0, 2]
def check_dataset(dataset):
    for sample in dataset:
        if sample['label'] not in [0, 1, 2]:
            print("Unexpected label value found:", sample['label'])
        if sample['input_ids'].dtype != torch.int64 or sample['attention_mask'].dtype != torch.int64:
            print("Data type mismatch found.")

# Check train, validation, and test datasets for any inconsistencies
check_dataset(train_tokenized)
check_dataset(val_tokenized)
check_dataset(test_tokenized)
print("Unique labels in training set:", torch.unique(train_tokenized['label']))


Unique labels in training set: tensor([0, 1, 2])


In [8]:
# Configure PEFT with LoRA (Low-Rank Adaptation)
lora_config = LoraConfig(
    r=16,  # Rank for LoRA
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none",
)

# Wrap the model with PEFT
model = get_peft_model(model, lora_config)

In [9]:
import torch
from torch.utils.data import DataLoader

# Assuming model and tokenizer are already defined and loaded
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)
tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2", trust_remote_code=True)

# Set padding token if not already set
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token  # Assuming eos_token can serve as padding

# Set pad_token_id for the model configuration
model.config.pad_token_id = tokenizer.pad_token_id

# Create a DataLoader for the test dataset
test_loader = DataLoader(test_tokenized, batch_size=4, shuffle=False)  # Adjust batch_size as needed

# Initialize variables for tracking predictions and labels
all_predictions = []
all_labels = []

# Set the model to evaluation mode
model.eval()

# Disable gradient calculation
with torch.no_grad():
    for batch in test_loader:
        # Move the batch to the GPU
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)

        # Forward pass
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)

        # Get logits and predictions
        logits = outputs.logits if hasattr(outputs, 'logits') else outputs[0]
        predictions = torch.argmax(logits, dim=-1)

        # Store predictions and labels if available
        all_predictions.append(predictions.cpu())  # Move predictions to CPU
        if 'label' in batch:  # Ensure 'label' is the correct key in your dataset
            all_labels.append(batch['label'].cpu())  # Store true labels

# Concatenate predictions and labels
all_predictions = torch.cat(all_predictions)
if all_labels:
    all_labels = torch.cat(all_labels)

    # Calculate accuracy only if true labels are present
    accuracy = (all_predictions == all_labels).float().mean().item()
    print(f"Accuracy on test set: {accuracy:.4f}")

    # Additional evaluation metrics can be calculated here (e.g., F1 score, precision, recall)

print(f"Pre trained model accuracy: {accuracy:.4f}")

We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)


Accuracy on test set: 0.3400
Pre trained model accuracy: 0.3400


In [18]:
# Set training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=4,
    fp16=True,
    num_train_epochs=5,
    weight_decay=0.01,
    save_total_limit=5,
    logging_dir='./logs',
    save_strategy='epoch',
    resume_from_checkpoint=True,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [10]:
import zipfile
import os
import time
from transformers import TrainerCallback, Trainer, DataCollatorWithPadding, AutoModelForSequenceClassification, AutoTokenizer
import gc
import torch

# Base directory to save all checkpoints
checkpoint_base_dir = './checkpoints'

# Define a callback to save and compress the model after each epoch
class SaveAndCompressCallback(TrainerCallback):
    def on_epoch_end(self, args, state, control, **kwargs):
        model_dir = os.path.join(checkpoint_base_dir, f'final_model_epoch_{int(state.epoch)}')
        os.makedirs(model_dir, exist_ok=True)

        # Save the model and tokenizer
        kwargs['model'].save_pretrained(model_dir)
        tokenizer.save_pretrained(model_dir)

        # Compress the checkpoint directory
        zip_path = f'{model_dir}.zip'
        with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
            for root, dirs, files in os.walk(model_dir):
                for file in files:
                    file_path = os.path.join(root, file)
                    zipf.write(file_path, os.path.relpath(file_path, model_dir))

        print(f"Checkpoint for epoch {int(state.epoch)} saved and compressed as '{zip_path}'.")

# Function to load a specific checkpoint by epoch and extract it
def get_specific_checkpoint(directory, epoch):
    zip_path = os.path.join(directory, f'final_model_epoch_{epoch}.zip')
    extracted_dir = zip_path.replace('.zip', '')

    # Extract the zip file to disk if it hasn't been extracted already
    if os.path.exists(zip_path) and not os.path.exists(extracted_dir):
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(extracted_dir)
        print(f"Extracted checkpoint: {extracted_dir}")

    return extracted_dir if os.path.exists(extracted_dir) else None

# Load model and tokenizer from a specific epoch checkpoint with reduced memory usage
specific_epoch = 4
resume_checkpoint = get_specific_checkpoint(checkpoint_base_dir, specific_epoch)
if resume_checkpoint:
    print(f"Loading model from checkpoint of epoch {specific_epoch}: {resume_checkpoint}")
    model = AutoModelForSequenceClassification.from_pretrained(
        resume_checkpoint, torch_dtype=torch.float16  # use float16 to save memory
    )
    tokenizer = AutoTokenizer.from_pretrained(resume_checkpoint)
else:
    print("No checkpoint found for the specified epoch. Starting with a fresh model.")
    model = AutoModelForSequenceClassification.from_pretrained('microsoft/phi-2', torch_dtype=torch.float16)
    tokenizer = AutoTokenizer.from_pretrained('microsoft/phi-2')  # replace with your model base

# Clear memory by deleting variables that are no longer needed
gc.collect()

# Create a data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Extracted checkpoint: ./checkpoints/final_model_epoch_4
Loading model from checkpoint of epoch 4: ./checkpoints/final_model_epoch_4


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of PhiForSequenceClassification were not initialized from the model checkpoint at microsoft/phi-2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Create a Trainer with the callback
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=val_tokenized,
    data_collator=data_collator,
    callbacks=[SaveAndCompressCallback()],  # Add callback to Trainer
)
for param in model.parameters():
    param.requires_grad = True

# # Start training and time the process, treating it as a fresh start from loaded model weights
start_time = time.time()
trainer.train()
end_time = time.time()

# # Calculate time taken
time_taken = end_time - start_time
print(f"Time taken for fine-tuning: {time_taken:.2f} seconds")

In [11]:
print(f"Test input shape: {test_tokenized['input_ids'].shape}")
print(f"Test label shape: {test_tokenized['label'].shape}")

Test input shape: torch.Size([100, 512])
Test label shape: torch.Size([100])


In [13]:
import torch
from torch.utils.data import DataLoader

# Assuming model and tokenizer are already defined and loaded
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)
tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2", trust_remote_code=True)

# Set padding token if not already set
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token  # Assuming eos_token can serve as padding

# Set pad_token_id for the model configuration
model.config.pad_token_id = tokenizer.pad_token_id

# Create a DataLoader for the test dataset
test_loader = DataLoader(test_tokenized, batch_size=4, shuffle=False)  # Adjust batch_size as needed

# Initialize variables for tracking predictions and labels
all_predictions = []
all_labels = []

# Set the model to evaluation mode
model.eval()

# Disable gradient calculation
with torch.no_grad():
    for batch in test_loader:
        # Move the batch to the GPU
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)

        # Forward pass
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)

        # Get logits and predictions
        logits = outputs.logits if hasattr(outputs, 'logits') else outputs[0]
        predictions = torch.argmax(logits, dim=-1)

        # Store predictions and labels if available
        all_predictions.append(predictions.cpu())  # Move predictions to CPU
        if 'label' in batch:  # Ensure 'label' is the correct key in your dataset
            all_labels.append(batch['label'].cpu())  # Store true labels

# Concatenate predictions and labels
all_predictions = torch.cat(all_predictions)
if all_labels:
    all_labels = torch.cat(all_labels)

    # Calculate accuracy only if true labels are present
    accuracy = (all_predictions == all_labels).float().mean().item()
    print(f"Accuracy on test set: {accuracy:.4f}")

    # Additional evaluation metrics can be calculated here (e.g., F1 score, precision, recall)

print(f"Fine-tuned model accuracy: {accuracy:.4f}")

# Total parameters and number of parameters fine-tuned
total_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters: {total_params}")


Accuracy on test set: 0.2500
Fine-tuned model accuracy: 0.2500
Total parameters: 2666915840


In [15]:
# Function to perform inference on a single pair of premise and hypothesis
def infer(premise, hypothesis):
    # Tokenize the inputs
    inputs = tokenizer(premise, hypothesis, truncation=True, padding=True, max_length=512, return_tensors='pt')

    # Move input tensors to the correct device
    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)

    # Perform inference
    model.eval()  # Set model to evaluation mode
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits if hasattr(outputs, 'logits') else outputs[0]
        prediction = torch.argmax(logits, dim=-1).item()  # Get the predicted class

    return prediction
# Chat-like inference loop
print("Welcome to the inference chat! Type 'exit' to end the chat.")
premise = input("Enter premise: ")

hypothesis = input("Enter hypothesis: ")

# Get the model's prediction
prediction = infer(premise, hypothesis)

# Map the prediction to a human-readable label
label_mapping = {0: 'Entailment', 1: 'Neutral', 2: 'Contradiction'}
result = label_mapping.get(prediction, 'Unknown')

print(f"Prediction: {result}\n")

Welcome to the inference chat! Type 'exit' to end the chat.
Enter premise: A person on a horse jumps over a broken down airplane.
Enter hypothesis: A person is at a diner, ordering an omelette.
Prediction: Neutral

